In [13]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsRegressor
from math import sqrt

# Đọc dữ liệu người dùng
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

n_users = users.shape[0]
print("Number of users:", n_users)
# users.head()  # Uncomment để xem một vài ví dụ

# Đọc dữ liệu đánh giá
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

print(ratings_base.head(20))
print('Number of training rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])

# Đọc dữ liệu phim
i_cols = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime',
          'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
          'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

n_items = items.shape[0]
print("Number of items:", n_items)

X0 = items.to_numpy()
X_train_counts = X0[:, -19:]  # Các thể loại phim (19 cột cuối)

# Chuyển đổi các thể loại phim thành TF-IDF
transformer = TfidfTransformer(smooth_idf=True, norm='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

Number of users: 943
    user_id  movie_id  rating  unix_timestamp
0         1         1       5       874965758
1         1         2       3       876893171
2         1         3       4       878542960
3         1         4       3       876893119
4         1         5       3       889751712
5         1         6       5       887431973
6         1         7       4       875071561
7         1         8       1       875072484
8         1         9       5       878543541
9         1        10       3       875693118
10        1        11       2       875072262
11        1        12       5       878542960
12        1        13       5       875071805
13        1        14       5       874965706
14        1        15       5       875071608
15        1        16       5       878543541
16        1        17       3       875073198
17        1        18       4       887432020
18        1        19       5       875071515
19        1        21       1       878542772
Number of tra

In [14]:
print("TF-IDF của phim đầu tiên:", tfidf[0])

TF-IDF của phim đầu tiên: [0.         0.         0.         0.74066017 0.57387209 0.34941857
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [15]:


# Hàm lấy các phim được đánh giá bởi người dùng
def get_items_rated_by_user(rate_matrix, user_id):
    """
    Trả về (item_ids, scores)
    """
    y = rate_matrix[:, 0]  # tất cả người dùng
    # Các chỉ số phim được user_id đánh giá
    # Cần +1 vì trong rate_matrix, id bắt đầu từ 1 nhưng trong Python chỉ số bắt đầu từ 0
    ids = np.where(y == user_id + 1)[0]
    item_ids = rate_matrix[ids, 1] - 1  # chỉ số bắt đầu từ 0
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

# Đặt số lượng hàng xóm cho KNN
k_neighbors = 5  # Bạn có thể điều chỉnh giá trị này

# Khởi tạo danh sách để lưu trữ mô hình KNN cho mỗi người dùng
knn_models = [None] * n_users

# Huấn luyện KNN Regressor cho mỗi người dùng
for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    
    if len(ids) == 0:
        # Nếu người dùng chưa đánh giá phim nào, để None hoặc sử dụng giá trị mặc định
        knn_models[n] = None
        continue
    
    # Sử dụng KNN Regressor
    knn = KNeighborsRegressor(n_neighbors=k_neighbors, weights='uniform', metric='cosine')
    
    Xhat = tfidf[ids, :]  # Đặc trưng của các phim đã được đánh giá
    y = scores  # Điểm đánh giá thực tế
    
    # Fit mô hình KNN
    knn.fit(Xhat, y)
    
    # Lưu trữ mô hình
    knn_models[n] = knn

# Dự đoán điểm đánh giá cho tất cả người dùng và phim
Yhat = np.zeros((n_items, n_users))

for n in range(n_users):
    if knn_models[n] is not None:
        # Dự đoán điểm đánh giá cho tất cả các phim đối với người dùng n
        Yhat[:, n] = knn_models[n].predict(tfidf)
    else:
        # Nếu người dùng chưa đánh giá phim nào, gán giá trị trung bình hoặc 0
        Yhat[:, n] = 0  # Bạn có thể thay đổi thành giá trị khác như np.nan hoặc trung bình

# Kiểm tra dự đoán cho một người dùng cụ thể
user_id = 10  # ID người dùng trong bộ kiểm tra (giả sử 0-based index)
ids, scores = get_items_rated_by_user(rate_test, user_id)

if len(ids) > 0:
    print("Rated movies ids:", ids)
    print("True ratings:", scores)
    print("Predicted ratings:", Yhat[ids, user_id])
else:
    print(f"Người dùng ID {user_id} chưa đánh giá phim nào trong bộ kiểm tra.")

# Hàm đánh giá RMSE
def evaluate(Yhat, rates):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        if len(ids) == 0:
            continue  # Nếu người dùng chưa đánh giá phim nào, bỏ qua
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e * e).sum()
        cnt += e.size 
    return sqrt(se / cnt) if cnt != 0 else float('nan')

print("RMSE for training:", evaluate(Yhat, rate_train))
print("RMSE for test    :", evaluate(Yhat, rate_test))


Rated movies ids: [ 37 109 110 226 424 557 722 724 731 739]
True ratings: [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [4.  3.2 3.4 3.4 3.4 3.6 3.6 3.4 3.4 4. ]
RMSE for training: 0.9378686791172175
RMSE for test    : 1.088378793769645
